In [1]:
import os
import sys

sys.path.append(os.getcwd() + '/..')

from importance_driven_coverage import attributors, clusterers, coverage

%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms.v2 as T
from torch import nn
from torch.utils import data
from torch.utils.data import DataLoader


class LeNet5(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.conv1 = nn.Conv2d(1, 6, 5, padding=2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(400, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = F.max_pool2d(F.relu(self.conv1(x)), 2, 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2, 2)

        x = x.view(-1, 16*5*5)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LeNet5().to(device)

In [4]:
transforms = T.Compose([
    T.ToImage(),
    T.ToPureTensor(),
    T.ToDtype(torch.float32, scale=True)
])
train_dataset = torchvision.datasets.MNIST("data", download=True, transform=transforms, train=True)
train, validation = data.random_split(train_dataset, [0.9, 0.1])
train_loader = data.DataLoader(train, batch_size=128, num_workers=4)
val_loader = data.DataLoader(validation, batch_size=128, num_workers=4)

test_dataset = torchvision.datasets.MNIST("data", download=True, transform=transforms, train=True)
test_loader = data.DataLoader(test_dataset, batch_size=128, num_workers=4)

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adadelta(model.parameters(), lr=1.0, rho=0.95)

def train(dataloader, model, criterion, opt):
        size = len(dataloader.dataset)
        num_batches = len(dataloader)
        model.train()
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            pred = model(X)
            loss = criterion(pred, y)

            loss.backward()
            opt.step()
            opt.zero_grad()

            if batch % (num_batches//10) == 0:
                loss, current = loss.item(), (batch + 1) * len(X)
                print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def val(dataloader, model, criterion):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += criterion(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, criterion, optimizer)
    val(val_loader, model, criterion)
print("Done!")

Epoch 1
-------------------------------


loss: 2.305979 [  128/54000]
loss: 0.958432 [ 5504/54000]
loss: 0.255655 [10880/54000]
loss: 0.279248 [16256/54000]
loss: 0.206301 [21632/54000]
loss: 0.048889 [27008/54000]
loss: 0.115305 [32384/54000]
loss: 0.133916 [37760/54000]
loss: 0.048265 [43136/54000]
loss: 0.077177 [48512/54000]
loss: 0.022363 [53888/54000]
Test Error: 
 Accuracy: 97.6%, Avg loss: 0.080670 

Epoch 2
-------------------------------
loss: 0.022842 [  128/54000]
loss: 0.129321 [ 5504/54000]
loss: 0.118705 [10880/54000]
loss: 0.065015 [16256/54000]
loss: 0.146484 [21632/54000]
loss: 0.020892 [27008/54000]
loss: 0.023454 [32384/54000]
loss: 0.142362 [37760/54000]
loss: 0.008944 [43136/54000]
loss: 0.090765 [48512/54000]
loss: 0.012042 [53888/54000]
Test Error: 
 Accuracy: 97.6%, Avg loss: 0.075657 

Epoch 3
-------------------------------
loss: 0.021661 [  128/54000]
loss: 0.034378 [ 5504/54000]
loss: 0.075865 [10880/54000]
loss: 0.043923 [16256/54000]
loss: 0.128253 [21632/54000]
loss: 0.025787 [27008/54000]
loss

In [6]:
layer = model.fc3
n = 8

attributor = attributors.CaptumLRPAttributor(attribute_kwargs={"attribute_to_layer_input": True})
clusterer = clusterers.KMeansClustererSimpleSilhouette()

idc = coverage.ImportanceDrivenCoverage(model, attributor, clusterer)

score, cov = idc.calculate(train_loader, test_loader, layer, n, layer_input=True,
                           attributions_path="attributions.pt",
                           centroids_path="centroids.pt")

/home/ybbat/.micromamba/envs/idc/lib/python3.11/site-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(


In [7]:
new_trans = transforms = T.Compose([
    T.RandAugment()
])
score_after_trans, cov_after_trans = idc.calculate(train_loader, test_loader, layer, n, transform=new_trans, layer_input=True,
                                                   attributions_path="attributions.pt",
                                                   centroids_path="centroids.pt")

In [8]:
print(f"{len(idc.total_combs)}")

65536


In [9]:
print(f"Score without transform: {score}")
print(f"Score with transform: {score}")
print(f"Combinations gained from transform: {len(cov_after_trans - cov)}")

Score without transform: 0.02386474609375
Score with transform: 0.02386474609375
Combinations gained from transform: 484
